In [1]:
import torch

In [5]:
outputs = torch.load("../data/outputs.pt", map_location="cpu", weights_only=True)

In [7]:
outputs[0]

{'logits': tensor([[ -3.8360,  -3.8213,   0.2260,  ...,  -3.8360,  -3.8362,  -3.8359],
         [-10.3110, -10.3242,  -2.9179,  ..., -10.3110, -10.3106, -10.3109],
         [-10.3623, -10.4126,  -1.5401,  ..., -10.3623, -10.3626, -10.3624],
         ...,
         [ -7.4106,  -7.4040,   1.7439,  ...,  -7.4106,  -7.4120,  -7.4109],
         [ -6.5133,  -6.4513,   0.5443,  ...,  -6.5133,  -6.5144,  -6.5135],
         [ -6.8068,  -6.7512,   1.7199,  ...,  -6.8067,  -6.8073,  -6.8070]]),
 'hidden_states': None,
 'router_logits': (tensor([[ 1.2426, -0.6960, -0.1843,  ..., -0.0854, -0.3065, -0.8278],
          [ 0.0486, -0.6635, -0.2995,  ..., -0.8019, -0.2085, -0.3599],
          [ 0.2210, -0.2559, -0.1717,  ..., -0.1653,  0.1054, -0.3042],
          ...,
          [-0.3468, -0.9403, -0.5039,  ..., -0.7284, -0.6300, -0.7022],
          [-0.5475, -0.6101, -0.5218,  ...,  0.1284, -0.5403, -0.8605],
          [-0.0076, -0.3588, -0.1842,  ...,  1.0570, -0.3121, -1.3218]]),
  tensor([[-0.6127, -0